In [44]:
!pip install pymongo[srv,tls]

In [45]:
import pymongo
import json
from pprint import pprint

In [ ]:
mongo_user = "xxxxxxxxx"
mongo_pw = "xxxxxxxxx"
mongo_host = "xxxxxxxxx"
mongo_app = "xxxxxxxxx"

conexao = pymongo.MongoClient("mongodb+srv://" + mongo_user + ":" + mongo_pw + "@" + mongo_host + "/?retryWrites=true&w=majority&appName=" + mongo_app)

In [46]:
conexao.list_database_names()

['Trabalho1', 'admin', 'local']

In [47]:
bd_Trabalho1 = conexao["Trabalho1"]


In [48]:
print(conexao.list_database_names())

['Trabalho1', 'admin', 'local']


In [49]:
bd = conexao["Trabalho1"]

In [59]:
bd.list_collection_names()

[]

In [85]:
bd.drop_collection("clientes")
bd.drop_collection("produtos")
bd.drop_collection("pedidos")

{'nIndexesWas': 1,
 'ns': 'Trabalho1.pedidos',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1730420690, 2),
  'signature': {'hash': b'\x9a}I\xfd\x8fl\xa9\x95\xe1\x1f\x8e\xa4k\x08(gA\xda3\xa2',
   'keyId': 7426820941021184005}},
 'operationTime': Timestamp(1730420690, 2)}

In [86]:
bd.create_collection("clientes", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "title": "Clientes do estabelecimento",
        "required": ["nome", "data_cadastro", "tipo", "documento", "endereco", "telefones", "cliente_id"],
        "properties": {
            "nome": {
                "bsonType": "string",
                "description": "Nome do cliente, obrigatório e deve ser uma string"
            },
            "data_cadastro": {
                "bsonType": "date",
                "description": "Data de cadastro, obrigatória e deve ser uma data"
            },
            "tipo": {
                "bsonType": "string",
                "enum": ["F", "J"],
                "description": "Tipo de cliente, obrigatório e deve ser 'F' (Pessoa física) ou 'J' (Pessoa Jurídica)"
            },
            "documento": {
                "bsonType": "object",
                "description": "Informações de CPF ou CNPJ",
                "properties": {
                    "cpf": {
                        "bsonType": "string",
                        "pattern": "^[0-9]{3}\\.[0-9]{3}\\.[0-9]{3}-[0-9]{2}$",
                        "description": "CPF do cliente, obrigatório para pessoa física, formato XXX.XXX.XXX-XX"
                    },
                    "data_nascimento": {
                        "bsonType": "date",
                        "description": "Data de nascimento, obrigatória para pessoa física"
                    },
                    "genero": {
                        "bsonType": "string",
                        "enum": ["M", "F"],
                        "description": "Gênero, obrigatório para pessoa física"
                    },
                    "cnpj": {
                        "bsonType": "string",
                        "pattern": "^[0-9]{2}\\.[0-9]{3}\\.[0-9]{3}\\/[0-9]{4}-[0-9]{2}$",
                        "description": "CNPJ da empresa, obrigatório para pessoa jurídica, formato XX.XXX.XXX/XXXX-XX"
                    },
                    "razao_social": {
                        "bsonType": "string",
                        "description": "Razão social, obrigatória para pessoa jurídica"
                    }
                }
            },
            "endereco": {
                "bsonType": "array",
                "minItems": 1,
                "items": {
                    "bsonType": "object",
                    "required": ["rua", "numero", "cep", "cidade"],
                    "properties": {
                        "rua": {
                            "bsonType": "string",
                            "description": "Nome da rua, obrigatório e deve ser uma string"
                        },
                        "numero": {
                            "bsonType": "int",
                            "description": "Número, obrigatório e deve ser um inteiro"
                        },
                        "complemento": {
                            "bsonType": "string",
                            "description": "Complemento, opcional"
                        },
                        "cep": {
                            "bsonType": "string",
                            "pattern": "^[0-9]{5}-[0-9]{3}$",
                            "description": "CEP no formato XXXXX-XXX"
                        },
                        "cidade": {
                            "bsonType": "object",
                            "required": ["nome", "uf", "estado_nome", "regiao"],
                            "properties": {
                                "nome": {
                                    "bsonType": "string",
                                    "description": "Nome da cidade, obrigatório"
                                },
                                "uf": {
                                    "bsonType": "string",
                                    "pattern": "^[A-Z]{2}$",
                                    "description": "UF com 2 letras maiúsculas"
                                },
                                "estado_nome": {
                                    "bsonType": "string",
                                    "description": "Nome do estado, obrigatório"
                                },
                                "regiao": {
                                    "bsonType": "string",
                                    "enum": ["S", "SE", "CO", "NE", "N"],
                                    "description": "Região, obrigatório e deve ser um valor válido"
                                }
                            }
                        }
                    }
                }
            },
            "telefones": {
                "bsonType": "array",
                "minItems": 1,
                "items": {
                    "bsonType": "object",
                    "required": ["ddd", "numero"],
                    "properties": {
                        "ddd": {
                            "bsonType": "int",
                            "minimum": 10,
                            "maximum": 99,
                            "description": "DDD do telefone"
                        },
                        "numero": {
                            "bsonType": "int",
                            "minimum": 10000000,
                            "maximum": 999999999,
                            "description": "Número do telefone sem DDD"
                        }
                    }
                }
            },
            "cliente_id": {
                "bsonType": "string",
                "description": "ID do cliente, obrigatório e deve ser uma string"
            }
        }
    }
});

In [87]:
bd.create_collection( "produtos", validator={
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["nome", "data_lancamento", "importado", "preco", "prazo_entrega", "categoria", "produto_id"],
            "properties": {
                "nome": {
                    "bsonType": "string",
                    "description": "Nome do produto, obrigatório e deve ser uma string"
                },
                "data_lancamento": {
                    "bsonType": "date",
                    "description": "Data de lançamento, obrigatória e deve ser uma data"
                },
                "importado": {
                    "bsonType": "bool",
                    "description": "Indica se o produto é importado, obrigatório e deve ser booleano"
                },
                "preco": {
                    "bsonType": "double",
                    "minimum": 0,
                    "description": "Preço do produto, obrigatório e deve ser maior ou igual a zero"
                },
                "prazo_entrega": {
                    "bsonType": "int",
                    "minimum": 0,
                    "description": "Prazo de entrega em dias, obrigatório e deve ser um inteiro"
                },
                "categoria": {
                    "bsonType": "object",
                    "required": ["_id", "nome"],
                    "properties": {
                        "_id": {
                            "bsonType": "objectId",
                            "description": "ID da categoria, obrigatório"
                        },
                        "nome": {
                            "bsonType": "string",
                            "description": "Nome da categoria, obrigatório"
                        }
                    }
                },
                "avaliacoes": {
                    "bsonType": "array",
                    "items": {
                        "bsonType": "object",
                        "required": ["cliente_id", "nota"],
                        "properties": {
                            "cliente_id": {
                                "bsonType": "string",
                                "description": "ID do cliente que avaliou"
                            },
                            "nota": {
                                "bsonType": "int",
                                "minimum": 1,
                                "maximum": 5,
                                "description": "Nota da avaliação entre 1 e 5"
                            },
                            "comentario": {
                                "bsonType": "string",
                                "description": "Comentário da avaliação, opcional"
                            }
                        }
                    }
                },
                "produto_id": {
                    "bsonType": "string",
                    "description": "ID do produto, obrigatório e deve ser uma string"
                }
            }
        }
    }
);


In [88]:
bd.create_collection("pedidos", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["data_emissao", "cliente", "itens"],
        "properties": {
            "data_emissao": {
                "bsonType": "date",
                "description": "Data de emissão do pedido, obrigatória"
            },
            "cliente": {
                "bsonType": "object",
                "required": ["cliente_id", "nome"],
                "properties": {
                    "cliente_id": {
                        "bsonType": "string",
                        "description": "ID do cliente, obrigatório"
                    },
                    "nome": {
                        "bsonType": "string",
                        "description": "Nome do cliente, obrigatório"
                    }
                }
            },
            "itens": {
                "bsonType": "array",
                "minItems": 1,
                "items": {
                    "bsonType": "object",
                    "required": ["produto", "quantidade", "valor_unitario"],
                    "properties": {
                        "produto": {
                            "bsonType": "object",
                            "required": ["produto_id", "nome"],
                            "properties": {
                                "produto_id": {
                                    "bsonType": "string",
                                    "description": "ID do produto, obrigatório"
                                },
                                "nome": {
                                    "bsonType": "string",
                                    "description": "Nome do produto, obrigatório"
                                }
                            }
                        },
                        "quantidade": {
                            "bsonType": "int",
                            "minimum": 1,
                            "description": "Quantidade do produto, obrigatória e deve ser pelo menos 1"
                        },
                        "valor_unitario": {
                            "bsonType": "double",
                            "minimum": 0,
                            "description": "Valor unitário do produto, obrigatório e deve ser maior ou igual a zero"
                        },
                        "entrega": {
                            "bsonType": "object",
                            "required": ["num_entrega", "data", "veiculo"],
                            "properties": {
                                "num_entrega": {
                                    "bsonType": "int",
                                    "description": "Número da entrega, obrigatório"
                                },
                                "data": {
                                    "bsonType": "date",
                                    "description": "Data da entrega, obrigatória"
                                },
                                "veiculo": {
                                    "bsonType": "object",
                                    "required": ["placa", "motorista"],
                                    "properties": {
                                        "placa": {
                                            "bsonType": "string",
                                            "pattern": "^[A-Z]{3}-[0-9]{4}$",
                                            "description": "Placa do veículo"
                                        },
                                        "motorista": {
                                            "bsonType": "object",
                                            "required": ["cnh", "nome"],
                                            "properties": {
                                                "cnh": {
                                                    "bsonType": "string",
                                                    "pattern": "^[0-9]{11}$",
                                                    "description": "CNH do motorista"
                                                },
                                                "nome": {
                                                    "bsonType": "string",
                                                    "description": "Nome do motorista"
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
});

In [71]:
from datetime import datetime
from bson import ObjectId

In [89]:
dados = [
    {
        "cliente_id": "1",
        "nome": "Ana Costa",
        "data_cadastro": datetime.strptime("2023-02-10T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "tipo": "F",
        "documento": {
            "cpf": "321.654.987-01",
            "data_nascimento": datetime.strptime("1985-06-15T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
            "genero": "F"
        },
        "endereco": [
            {
                "rua": "Rua dos Jacarandás",
                "numero": 120,
                "cep": "87654-321",
                "cidade": {
                    "nome": "Curitiba",
                    "uf": "PR",
                    "estado_nome": "Paraná",
                    "regiao": "S"
                }
            }
        ],
        "telefones": [
            {
                "ddd": 41,
                "numero": 999998888
            }
        ]
    },
    {
        "cliente_id": "2",
        "nome": "Lucas Almeida",
        "data_cadastro": datetime.strptime("2023-03-25T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "tipo": "F",
        "documento": {
            "cpf": "123.987.654-32",
            "data_nascimento": datetime.strptime("1992-11-30T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
            "genero": "M"
        },
        "endereco": [
            {
                "rua": "Avenida das Palmeiras",
                "numero": 55,
                "cep": "11223-445",
                "cidade": {
                    "nome": "Florianópolis",
                    "uf": "SC",
                    "estado_nome": "Santa Catarina",
                    "regiao": "S"
                }
            }
        ],
        "telefones": [
            {
                "ddd": 48,
                "numero": 987654321
            }
        ]
    },
    {
        "cliente_id": "3",
        "nome": "Maria Silva",
        "data_cadastro": datetime.strptime("2023-04-12T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "tipo": "F",
        "documento": {
            "cpf": "456.789.123-45",
            "data_nascimento": datetime.strptime("1978-03-05T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
            "genero": "F"
        },
        "endereco": [
            {
                "rua": "Rua das Hortênsias",
                "numero": 256,
                "cep": "54321-098",
                "cidade": {
                    "nome": "Belo Horizonte",
                    "uf": "MG",
                    "estado_nome": "Minas Gerais",
                    "regiao": "SE"
                }
            }
        ],
        "telefones": [
            {
                "ddd": 31,
                "numero": 998877665
            }
        ]
    },
    {
        "cliente_id": "4",
        "nome": "Tech Solutions LTDA",
        "data_cadastro": datetime.strptime("2023-05-10T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "tipo": "J",
        "documento": {
            "cnpj": "12.345.678/0001-90",
            "razao_social": "Tech Solutions Tecnologia"
        },
        "endereco": [
            {
                "rua": "Rua do Comércio",
                "numero": 789,
                "cep": "98765-432",
                "cidade": {
                    "nome": "São Paulo",
                    "uf": "SP",
                    "estado_nome": "São Paulo",
                    "regiao": "SE"
                }
            }
        ],
        "telefones": [
            {
                "ddd": 11,
                "numero": 912345678
            }
        ]
    },
    {
        "cliente_id": "5",
        "nome": "Alimentos S/A",
        "data_cadastro": datetime.strptime("2023-06-20T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "tipo": "J",
        "documento": {
            "cnpj": "98.765.432/0001-12",
            "razao_social": "Alimentos e Bebidas S/A"
        },
        "endereco": [
            {
                "rua": "Avenida Central",
                "numero": 654,
                "cep": "23456-789",
                "cidade": {
                    "nome": "Rio de Janeiro",
                    "uf": "RJ",
                    "estado_nome": "Rio de Janeiro",
                    "regiao": "SE"
                }
            }
        ],
        "telefones": [
            {
                "ddd": 21,
                "numero": 923456789
            }
        ]
    }
]

bd.clientes.insert_many(dados)

InsertManyResult([ObjectId('67241fe0af14e54def4fa1f6'), ObjectId('67241fe0af14e54def4fa1f7'), ObjectId('67241fe0af14e54def4fa1f8'), ObjectId('67241fe0af14e54def4fa1f9'), ObjectId('67241fe0af14e54def4fa1fa')], acknowledged=True)

In [90]:
dados_produtos = [
    {
        "produto_id": "1",
        "nome": "Smartphone XPro",
        "data_lancamento": datetime.strptime("2023-01-15T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "importado": True,
        "preco": 2999.99,
        "prazo_entrega": 5,
        "categoria": {
            "_id": ObjectId("64b8d4e78b2e4c0b64f7a1f1"),
            "nome": "Eletrônicos"
        },
        "avaliacoes": [
            {
                "cliente_id": "1",
                "nota": 5,
                "comentario": "Ótimo desempenho e câmera excelente."
            }
        ]
    },
    {
        "produto_id": "2",
        "nome": "Smart TV 4K",
        "data_lancamento": datetime.strptime("2022-11-20T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "importado": True,
        "preco": 2599.99,
        "prazo_entrega": 7,
        "categoria": {
            "_id": ObjectId("64b8d4e78b2e4c0b64f7a1f2"),
            "nome": "Eletrônicos"
        },
        "avaliacoes": [
            {
                "cliente_id": "2",
                "nota": 4,
                "comentario": "Excelente qualidade de imagem."
            }
        ]
    },
    {
        "produto_id": "3",
        "nome": "Notebook Ultra",
        "data_lancamento": datetime.strptime("2023-02-10T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "importado": True,
        "preco": 4999.99,
        "prazo_entrega": 10,
        "categoria": {
            "_id": ObjectId("64b8d4e78b2e4c0b64f7a1f3"),
            "nome": "Computadores"
        },
        "avaliacoes": [
            {
                "cliente_id": "3",
                "nota": 5,
                "comentario": "Desempenho incrível e ótima construção."
            }
        ]
    },
    {
        "produto_id": "4",
        "nome": "Cafeteira Automática",
        "data_lancamento": datetime.strptime("2021-06-15T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "importado": False,
        "preco": 799.99,
        "prazo_entrega": 4,
        "categoria": {
            "_id": ObjectId("64b8d4e78b2e4c0b64f7a1f4"),
            "nome": "Eletrodomésticos"
        },
        "avaliacoes": [
            {
                "cliente_id": "4",
                "nota": 4,
                "comentario": "Faz um ótimo café, mas é um pouco barulhenta."
            }
        ]
    },
    {
        "produto_id": "5",
        "nome": "Fone de Ouvido Bluetooth",
        "data_lancamento": datetime.strptime("2023-03-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "importado": False,
        "preco": 199.99,
        "prazo_entrega": 3,
        "categoria": {
            "_id": ObjectId("64b8d4e78b2e4c0b64f7a1f5"),
            "nome": "Acessórios"
        },
        "avaliacoes": [
            {
                "cliente_id": "5",
                "nota": 4,
                "comentario": "Boa qualidade de som, mas poderia ter mais bateria."
            }
        ]
    }
]

bd.produtos.insert_many(dados_produtos)


InsertManyResult([ObjectId('67241fe5af14e54def4fa1fb'), ObjectId('67241fe5af14e54def4fa1fc'), ObjectId('67241fe5af14e54def4fa1fd'), ObjectId('67241fe5af14e54def4fa1fe'), ObjectId('67241fe5af14e54def4fa1ff')], acknowledged=True)

In [91]:
dados_pedidos = [
    {
        "data_emissao": datetime.strptime("2023-07-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "cliente": {
            "cliente_id": "1",  # ID Ana Costa
            "nome": "Ana Costa"
        },
        "itens": [
            {
                "produto": {
                    "produto_id": "1",  # ID do produto Smartphone XPro
                    "nome": "Smartphone XPro"
                },
                "quantidade": 1,
                "valor_unitario": 2999.99,
                "entrega": {
                    "num_entrega": 1,
                    "data": datetime.strptime("2023-07-05T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
                    "veiculo": {
                        "placa": "ABC-1234",
                        "motorista": {
                            "cnh": "12345678901",
                            "nome": "Carlos Silva"
                        }
                    }
                }
            }
        ]
    },
    {
        "data_emissao": datetime.strptime("2023-07-10T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "cliente": {
            "cliente_id": "2",  # ID do cliente Lucas Almeida
            "nome": "Lucas Almeida"
        },
        "itens": [
            {
                "produto": {
                    "produto_id": "2",  # ID do produto Smart TV 4K
                    "nome": "Smart TV 4K"
                },
                "quantidade": 1,
                "valor_unitario": 2599.99,
                "entrega": {
                    "num_entrega": 2,
                    "data": datetime.strptime("2023-07-15T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
                    "veiculo": {
                        "placa": "DEF-5678",
                        "motorista": {
                            "cnh": "23456789012",
                            "nome": "João Pereira"
                        }
                    }
                }
            }
        ]
    },
    {
        "data_emissao": datetime.strptime("2023-07-15T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "cliente": {
            "cliente_id": "3",  # ID do cliente Maria Silva
            "nome": "Maria Silva"
        },
        "itens": [
            {
                "produto": {
                    "produto_id": "3",  # ID do produto Notebook Ultra
                    "nome": "Notebook Ultra"
                },
                "quantidade": 2,
                "valor_unitario": 4999.99,
                "entrega": {
                    "num_entrega": 3,
                    "data": datetime.strptime("2023-07-20T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
                    "veiculo": {
                        "placa": "GHI-9101",
                        "motorista": {
                            "cnh": "34567890123",
                            "nome": "Ana Souza"
                        }
                    }
                }
            }
        ]
    },
    {
        "data_emissao": datetime.strptime("2023-07-20T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "cliente": {
            "cliente_id": "4",  # ID do cliente Tech Solutions LTDA
            "nome": "Tech Solutions LTDA"
        },
        "itens": [
            {
                "produto": {
                    "produto_id": "4",  # ID do produto Cafeteira Automática
                    "nome": "Cafeteira Automática"
                },
                "quantidade": 3,
                "valor_unitario": 799.99,
                "entrega": {
                    "num_entrega": 4,
                    "data": datetime.strptime("2023-07-25T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
                    "veiculo": {
                        "placa": "JKL-2345",
                        "motorista": {
                            "cnh": "45678901234",
                            "nome": "Pedro Lima"
                        }
                    }
                }
            }
        ]
    },
    {
        "data_emissao": datetime.strptime("2023-07-25T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
        "cliente": {
            "cliente_id": "5",  # ID do cliente Alimentos S/A
            "nome": "Alimentos S/A"
        },
        "itens": [
            {
                "produto": {
                    "produto_id": "5",  # ID do produto Fone de Ouvido Bluetooth
                    "nome": "Fone de Ouvido Bluetooth"
                },
                "quantidade": 5,
                "valor_unitario": 199.99,
                "entrega": {
                    "num_entrega": 5,
                    "data": datetime.strptime("2023-07-30T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"),
                    "veiculo": {
                        "placa": "MNO-6789",
                        "motorista": {
                            "cnh": "56789012345",
                            "nome": "Marcos Dias"
                        }
                    }
                }
            }
        ]
    }
]

bd.pedidos.insert_many(dados_pedidos)


InsertManyResult([ObjectId('67241fe9af14e54def4fa200'), ObjectId('67241fe9af14e54def4fa201'), ObjectId('67241fe9af14e54def4fa202'), ObjectId('67241fe9af14e54def4fa203'), ObjectId('67241fe9af14e54def4fa204')], acknowledged=True)

In [92]:
#listar os nomes dos clientes e as cidades onde residem;
clientes = bd.clientes.find({}, {"nome": 1, "endereco.cidade.nome": 1, "_id": 0})

for cliente in clientes:
    nome = cliente.get("nome")
    cidades = [end["cidade"]["nome"] for end in cliente["endereco"]]
    print(f"Cliente: {nome}, Cidades: {', '.join(cidades)}")



Cliente: Ana Costa, Cidades: Curitiba
Cliente: Lucas Almeida, Cidades: Florianópolis
Cliente: Maria Silva, Cidades: Belo Horizonte
Cliente: Tech Solutions LTDA, Cidades: São Paulo
Cliente: Alimentos S/A, Cidades: Rio de Janeiro


In [93]:
# listar a quantidade total de produtos em cada categoria;
pipeline = [
    {
        "$group": {
            "_id": "$categoria.nome",
            "total_produtos": {"$sum": 1}
        }
    },
    {
        "$sort": {"total_produtos": -1}
    }
]

resultado = bd.produtos.aggregate(pipeline)

for categoria in resultado:
    print(f"Categoria: {categoria['_id']}, Total de produtos: {categoria['total_produtos']}")


Categoria: Eletrônicos, Total de produtos: 2
Categoria: Computadores, Total de produtos: 1
Categoria: Acessórios, Total de produtos: 1
Categoria: Eletrodomésticos, Total de produtos: 1


In [94]:
#listar os produtos mais vendidos, ou seja, que aparecem no maior número de pedidos;
pipeline = [
    {
        "$unwind": "$itens"
    },
    {
        "$group": {
            "_id": "$itens.produto.nome",
            "total_vendas": {"$sum": 1}
        }
    },
    {
        "$sort": {"total_vendas": -1}
    }
]

resultado = bd.pedidos.aggregate(pipeline)

for produto in resultado:
    print(f"Produto: {produto['_id']}, Total de vendas: {produto['total_vendas']}")


Produto: Smartphone XPro, Total de vendas: 1
Produto: Notebook Ultra, Total de vendas: 1
Produto: Fone de Ouvido Bluetooth, Total de vendas: 1
Produto: Smart TV 4K, Total de vendas: 1
Produto: Cafeteira Automática, Total de vendas: 1


In [95]:
#listar o nome do cliente, número do pedido e o valor total do pedido em ordem decrescente de valor total.
pipeline = [
    {
        "$unwind": "$itens"
    },
    {
        "$group": {
            "_id": "$_id",
            "cliente_nome": {"$first": "$cliente.nome"},
            "valor_total": {"$sum": {"$multiply": ["$itens.quantidade", "$itens.valor_unitario"]}}
        }
    },
    {
        "$sort": {"valor_total": -1}
    }
]

resultado = bd.pedidos.aggregate(pipeline)

for pedido in resultado:
    print(f"Cliente: {pedido['cliente_nome']}, Número do pedido: {pedido['_id']}, Valor total: {pedido['valor_total']:.2f}")


Cliente: Maria Silva, Número do pedido: 67241fe9af14e54def4fa202, Valor total: 9999.98
Cliente: Ana Costa, Número do pedido: 67241fe9af14e54def4fa200, Valor total: 2999.99
Cliente: Lucas Almeida, Número do pedido: 67241fe9af14e54def4fa201, Valor total: 2599.99
Cliente: Tech Solutions LTDA, Número do pedido: 67241fe9af14e54def4fa203, Valor total: 2399.97
Cliente: Alimentos S/A, Número do pedido: 67241fe9af14e54def4fa204, Valor total: 999.95


In [102]:
pipeline = [
    {"$unwind": "$itens"},
    {"$lookup": {
        "from": "produtos",
        "localField": "itens.produto.produto_id",
        "foreignField": "produto_id",
        "as": "produto_info"
    }},
    {"$unwind": "$produto_info"},
    {"$group": {
        "_id": "$produto_info.categoria.nome",
        "total_vendas": {"$sum": 1}
    }},
    {"$sort": {"total_vendas": -1}},
    {"$limit": 3}
]

resultado = list(bd.pedidos.aggregate(pipeline))

for categoria in resultado:
    print(f"Categoria: {categoria['_id']}, Total de vendas: {categoria['total_vendas']}")



Categoria: Eletrônicos, Total de vendas: 2
Categoria: Eletrodomésticos, Total de vendas: 1
Categoria: Computadores, Total de vendas: 1
